# Comparative analysis description

The purpose of the analysis is to verify which of the graph embedding methods implemented in the ReLeGy package are applicable in graph visualization task. We compare whether the clusters of vertices are properly separated and whether the visual graph structure is preserved. The embeddings are visualized with t-SNE and PCA algorithms to allow for a broader perspective.

# Input data description

## Graphs 

* Barbell graph having two cliques of 20 vertices connected with a path of 5 vertices
* A cycle of 40 vertices
* A graph consisting of 5 cliques connected by random edges with density equal to 0.002 with 100 vertices in total
* A graph consisting of 7 cliques connected by random edges with density equal to 0.03 with 100 vertices in total
* A graph consisting of 7 clusters with density 0.8 connected by random edges with density equal to 0.03 with 100 vertices in total

## Embedding methods

* LaplacianEigenmaps
* GraphFactorization
* GraRep
* HOPE
* DeepWalk
* Node2Vec
* LINE
* HARP(DeepWalk)
* HARP(Node2Vec)
* Struc2Vec
* GraphWave

The GNN and GCN embedding methods are left out in this comparison, as they require vertices' labels and are not fit for a visualization task.

## Embedding dimensions

Each graph is embedded with each method three times, so that the resulting matrices have 2, 6, and 20 columns.

# Computation

In [1]:
import relegy.embeddings as rle
import relegy.graphs as rlr

In [15]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import inspect
from IPython.display import clear_output
from matplotlib.pylab import rcParams
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
import pickle
import re

In [55]:
embeddings = {}
for i in range(1, 6):
    for j in [2, 6, 20]:
        with open("G"+str(i)+"_results_vis_d"+str(j)+".p", mode="rb") as f:
            embeddings["G"+str(i)+"_"+"d"+str(j)] = pickle.load(f)

In [39]:
def get_embedding_methods_iterable():
    """
    Iterates over embedding methods.
    """
    return filter(lambda x: x[0][:2] != "__", inspect.getmembers(sys.modules['relegy.embeddings']))

In [5]:
#Graph generation
np.random.seed(123)
G1, labels1 = rlr.generate_graph("barbell", m1=20, m2=5), np.concatenate((np.repeat(1, 19), np.array([0, 2, 3, 4, 3, 2, 0]), np.repeat(1, 19)))
G2, labels2 = nx.cycle_graph(40), np.repeat(1, 40)
G3, labels3 = rlr.generate_clusters_graph(100, 5, 0.002, 1)
G4, labels4 = rlr.generate_clusters_graph(100, 7, 0.03, 1)
G5, labels5 = rlr.generate_clusters_graph(100, 7, 0.03, 0.8)

In [6]:
labels = [labels1, labels2, labels3, labels4, labels5]

In [7]:
#Graph plotting 
rcParams["figure.figsize"] = 8, 8
def plot_graph(graph):
    nx.draw(graph[0], node_color=graph[1])

dpdown = widgets.Dropdown(options = [('G1', (G1, labels1))], value=(G1, labels1), description="Graph: ")
with dpdown.hold_trait_notifications():
    dpdown.options=[('G1', (G1, labels1)), 
                           ('G2', (G2, labels2)), 
                           ('G3', (G3, labels3)), 
                           ('G4', (G4, labels4)), 
                           ('G5', (G5, labels5))]

interact(plot_graph, graph = dpdown)
plt.show()


interactive(children=(Dropdown(description='Graph: ', options=(('G1', (<networkx.classes.graph.Graph object at…

In [37]:
def get_results_from_all_viable_methods(d, graphs, graph_names=None):
    results = [None] * len(graphs)
    for i, G in enumerate(graphs):
        results_dictionary = {}
        for name, class_handle in get_embedding_methods_iterable():
            info = "Currently processing " + str(name) + ", d: " + str(d)
            if graph_names is not None:
                info += ", graph: " + graph_names[i]
            if not (name == "GCN" or name == "GNN"):
                if name in ["GraphWave", "HOPE", "LINE"]:
                    current_d = d // 2
                else:
                    current_d = d
                print(info)
                if not (name == "HARP"):
                    results_dictionary[name] = class_handle.fast_embed(G, d=current_d)
                    clear_output()
                else:
                    temp_name = "HARP_Deepwalk"
                    results_dictionary[temp_name] = class_handle.fast_embed(G, d=current_d)
                    temp_name = "HARP_Node2Vec"
                    results_dictionary[temp_name] = class_handle.fast_embed(G, d=current_d, method = "Node2Vec")
                    clear_output()
        results[i] = results_dictionary
    return results

In [11]:
#embeddings_d2 = get_results_from_all_viable_methods(d=2, graphs=[G1, G2, G3, G4, G5], graph_names=["G"+str(i) for i in range(1, 6)])

In [27]:
#embeddings_d6 = get_results_from_all_viable_methods(d=6, graphs=[G1, G2, G3, G4, G5], graph_names=["G"+str(i) for i in range(1, 6)])

In [40]:
#embeddings_d20 = get_results_from_all_viable_methods(d=20, graphs=[G1, G2, G3, G4, G5], graph_names=["G"+str(i) for i in range(1, 6)])

In [77]:
embeddings_d2 = [None] * 5
embeddings_d6 = [None] * 5
embeddings_d20 = [None] * 5
k = 0
l = 0
m = 0
for i in embeddings.keys():
    if re.search("d2$", i) is not None:
        embeddings_d2[k] = embeddings[i]
        k += 1
    elif "d6" in i:
        embeddings_d6[l] = embeddings[i]
        l += 1
    elif "d20" in i:
        embeddings_d20[m] = embeddings[i]
        m += 1

In [81]:
def plot_embeddings(embeddings):
    rcParams["figure.figsize"] = 16, 16
    fig, axs = plt.subplots(4, 4)
    names = list(embeddings[0].keys())
    for i in range(len(names)):
        ix_x = i // 4
        ix_y = i % 4
        cur_Z = embeddings[0][names[i]]
        if cur_Z.shape[1] != 2:
            ss = StandardScaler().fit_transform(cur_Z)
            pca_Z = PCA(n_components=2).fit_transform(cur_Z)
            cur_Z = pca_Z
        axs[ix_x, ix_y].scatter(cur_Z[:, 0], cur_Z[:, 1], c=embeddings[1])
        axs[ix_x, ix_y].set_title(names[i])
        axs[ix_x, ix_y].set_xticks([])
        axs[ix_x, ix_y].set_yticks([])
    while(i < 15):
        i += 1
        ix_x = i // 4
        ix_y = i % 4
        axs[ix_x, ix_y].axis('off')
    plt.show()
        
dpdown = widgets.Dropdown(options=[('G1', (embeddings_d2[0], labels[0]))],
                                    value=(embeddings_d2[0], labels[0]),
                                    description="Graph: ")
with dpdown.hold_trait_notifications():
    dpdown.options = [('G1', (embeddings_d2[0], labels[0])), 
                       ('G2', (embeddings_d2[1], labels[1])), 
                       ('G3', (embeddings_d2[2], labels[2])), 
                       ('G4', (embeddings_d2[3], labels[3])), 
                       ('G5', (embeddings_d2[4], labels[4]))]
interact(plot_embeddings, embeddings = dpdown)
plt.show()

interactive(children=(Dropdown(description='Graph: ', options=(('G1', ({'DNGR': array([[1.8253922e-04, 9.82244…

In [17]:
def plot_embeddings(embeddings):
    rcParams["figure.figsize"] = 16, 16
    fig, axs = plt.subplots(4, 4)
    names = list(embeddings[0].keys())
    for i in range(len(names)):
        ix_x = i // 4
        ix_y = i % 4
        cur_Z = embeddings[0][names[i]]
        if cur_Z.shape[1] != 2:
            ss = StandardScaler().fit_transform(cur_Z)
            pca_Z = PCA(n_components=2).fit_transform(cur_Z)
            cur_Z = pca_Z
        axs[ix_x, ix_y].scatter(cur_Z[:, 0], cur_Z[:, 1], c=embeddings[1])
        axs[ix_x, ix_y].set_title(names[i])
        axs[ix_x, ix_y].set_xticks([])
        axs[ix_x, ix_y].set_yticks([])
    while(i < 15):
        i += 1
        ix_x = i // 4
        ix_y = i % 4
        axs[ix_x, ix_y].axis('off')
    plt.show()
        
dpdown = widgets.Dropdown(options=[('G1', (embeddings_d6[0], labels[0]))],
                                    value=(embeddings_d6[0], labels[0]),
                                    description="Graph: ")
with dpdown.hold_trait_notifications():
    dpdown.options = [('G1', (embeddings_d6[0], labels[0])), 
                       ('G2', (embeddings_d6[1], labels[1])), 
                       ('G3', (embeddings_d6[2], labels[2])), 
                       ('G4', (embeddings_d6[3], labels[3])), 
                       ('G5', (embeddings_d6[4], labels[4]))]
interact(plot_embeddings, embeddings = dpdown)
plt.show()

interactive(children=(Dropdown(description='Graph: ', options=(('G1', ({'DNGR': array([[1.0000000e+00, 9.83646…

In [92]:
def plot_embeddings(embeddings):
    rcParams["figure.figsize"] = 16, 16
    fig, axs = plt.subplots(4, 4)
    names = list(embeddings[0].keys())
    for i in range(len(names)):
        ix_x = i // 4
        ix_y = i % 4
        cur_Z = embeddings[0][names[i]]
        if cur_Z.shape[1] != 2:
            ss = StandardScaler().fit_transform(cur_Z)
            pca_Z = TSNE(n_components=2).fit_transform(cur_Z)
            cur_Z = pca_Z
        axs[ix_x, ix_y].scatter(cur_Z[:, 0], cur_Z[:, 1], c=embeddings[1])
        axs[ix_x, ix_y].set_title(names[i])
        axs[ix_x, ix_y].set_xticks([])
        axs[ix_x, ix_y].set_yticks([])
    while(i < 15):
        i += 1
        ix_x = i // 4
        ix_y = i % 4
        axs[ix_x, ix_y].axis('off')
    plt.show()
        
dpdown = widgets.Dropdown(options=[('G1', (embeddings_d20[0], labels[0]))],
                                    value=(embeddings_d20[0], labels[0]),
                                    description="Graph: ")
with dpdown.hold_trait_notifications():
    dpdown.options = [('G1', (embeddings_d20[0], labels[0])), 
                       ('G2', (embeddings_d20[1], labels[1])), 
                       ('G3', (embeddings_d20[2], labels[2])), 
                       ('G4', (embeddings_d20[3], labels[3])), 
                       ('G5', (embeddings_d20[4], labels[4]))]
interact(plot_embeddings, embeddings = dpdown)
plt.show()

interactive(children=(Dropdown(description='Graph: ', options=(('G1', ({'DNGR': array([[4.65160042e-01, 3.4023…

In [91]:
(embeddings_d20[3], labels[3])

({'DNGR': array([[0.8182093 , 0.77744675, 0.11112484, ..., 0.45667663, 0.20318374,
          0.8991232 ],
         [0.87076795, 0.7079951 , 0.07771802, ..., 0.6241195 , 0.24355948,
          0.8392889 ],
         [0.8903668 , 0.7702857 , 0.04713255, ..., 0.5705624 , 0.23132947,
          0.831133  ],
         ...,
         [0.31148326, 0.00865889, 0.88679934, ..., 0.4155929 , 0.7713163 ,
          0.36114317],
         [0.19970727, 0.01016885, 0.8123678 , ..., 0.6256654 , 0.59770036,
          0.48113164],
         [0.32317346, 0.00772291, 0.8995787 , ..., 0.47268453, 0.7584353 ,
          0.40478438]], dtype=float32),
  'DeepWalk': array([[-0.16149025,  0.0200223 , -1.0025679 , ..., -0.15187481,
           0.5784219 ,  0.02054436],
         [ 0.5642022 , -0.22605135, -0.56505793, ..., -0.24931958,
          -0.17960708, -0.1989695 ],
         [ 0.411328  , -0.3445765 , -0.86157155, ...,  0.09695203,
           0.28699073,  0.08179238],
         ...,
         [-0.08075514,  0.03517565,